## Rhetorical relations classification used in tree building: Step 1. Data preparation

Make train/dev/test splitting, save in the corresponding .pkl files

Output:
 - ``data_labeling/*``

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.train_test_split import split_rstreebank

print('Loading RSTreebank:')
train, dev, test = split_rstreebank('./data_ru')
print('Train length:', len(train), 'Dev length:', len(dev), 'Test length:', len(test), '(files)')

In [ ]:
import pandas as pd
from utils.file_reading import read_gold

random_state = 45

train_samples = []
test_samples = []
dev_samples = []

for file in train:
    train_samples.append(read_gold(file.replace('.edus', ''), features=True))

for file in dev:
    dev_samples.append(read_gold(file.replace('.edus', ''), features=True))
    
for file in test:
    test_samples.append(read_gold(file.replace('.edus', ''), features=True))

train_samples = pd.concat(train_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
dev_samples = pd.concat(dev_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)
test_samples = pd.concat(test_samples).sample(frac=1, random_state=random_state).reset_index(drop=True)

In [ ]:
test_samples.category_id.unique()

In [ ]:
import os

def correct_samples(row):
    if row.snippet_x[0] in (',', '.', '!', '?'):
        row.snippet_x = row.snippet_x[1:].strip()
    if row.snippet_y[0] in (',', '.'):
        row.snippet_x += row.snippet_y[0]
        row.snippet_y = row.snippet_y[1:].strip()
    return row


OUT_PATH = 'data_labeling'
! mkdir $OUT_PATH
train_samples.to_pickle(os.path.join(OUT_PATH, 'train_samples.pkl'))
dev_samples.to_pickle(os.path.join(OUT_PATH, 'dev_samples.pkl'))
test_samples.to_pickle(os.path.join(OUT_PATH, 'test_samples.pkl'))

In [ ]:
counts = train_samples['relation'].value_counts(normalize=False).values
counts

In [ ]:
all_data = pd.concat([train_samples, dev_samples, test_samples])

In [ ]:
all_data['relation'].value_counts(normalize=False)

In [ ]:
dev_samples['relation'].value_counts()